# Chapter 4 Exercise 1: 表示学习基础 - 为深度因果模型铺路

## 学习目标

1. 理解为什么需要学习表示 (Representation Learning)
2. 掌握简单的神经网络特征提取
3. 理解处理组和对照组表示的差异
4. 为深度因果模型打下基础

---

## 从传统机器学习到深度因果模型

### 传统方法的局限

我们之前学习的 S-Learner、T-Learner 等方法都依赖于 **手工特征**。但现实世界中：

- 特征可能是非线性的
- 真正有用的信息可能隐藏在特征的复杂组合中
- 高维数据 (如图像、文本) 无法直接使用

### 表示学习的魔力

**表示学习 (Representation Learning)** 让神经网络自动学习最有用的特征表示：

$$X \xrightarrow{\text{神经网络}} \Phi(X) \xrightarrow{\text{因果模型}} Y(0), Y(1)$$

这个 $\Phi(X)$ 就是学习到的 "表示"！

## 生活化类比：翻译器

想象你是一个只懂中文的人，要判断一篇英文论文的质量。

**传统方法**：
- 手工提取特征：单词数、句子长度、标点符号数...
- 这些特征可能和论文质量关系不大

**表示学习方法**：
- 先用神经网络把英文 "翻译" 成一种通用表示
- 这个表示捕获了论文的语义信息
- 然后用这个表示来预测论文质量

神经网络就像一个 **智能翻译器**，把原始数据翻译成对任务最有用的形式！

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from typing import Tuple
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# 设置随机种子
np.random.seed(42)
torch.manual_seed(42)

print(f"PyTorch 版本: {torch.__version__}")
print("环境准备完成！")

---

## Part 1: 非线性数据的挑战

### 为什么线性模型不够用?

假设真实的数据生成过程是非线性的：

- 原始特征: $X_1, X_2 \sim N(0, 1)$
- 真正有用的特征: $\Phi_1 = \sin(X_1)$, $\Phi_2 = X_1 \times X_2$
- 处理分配: $P(T=1|X) = \sigma(\Phi_1 + \Phi_2)$
- 结果: $Y = 1 + 2T + \Phi_1 + 0.5\Phi_2 + \epsilon$

如果我们只用 $X_1, X_2$ 建模，会错过关键的非线性关系！

In [ ]:
# 练习 1.1: 生成非线性数据

def generate_nonlinear_data(
    n: int = 1000,
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    生成非线性数据: 原始特征无法直接捕获因果关系
    
    DGP (Data Generating Process):
    - X1, X2 ~ N(0, 1)
    - 真实有用特征: Phi1 = sin(X1), Phi2 = X1 * X2
    - T ~ Bernoulli(sigmoid(Phi1 + Phi2))
    - Y = 1 + 2*T + Phi1 + 0.5*Phi2 + noise
    
    TODO: 完成数据生成
    
    Returns:
        (X, T, Y) - 特征矩阵、处理、结果
    """
    np.random.seed(seed)
    
    # TODO: 生成原始特征 X1, X2
    X1 = None  # 你的代码: np.random.randn(n)
    X2 = None  # 你的代码: np.random.randn(n)
    
    # TODO: 计算有用特征
    # Phi1 = sin(X1), Phi2 = X1 * X2
    Phi1 = None  # 你的代码: np.sin(X1)
    Phi2 = None  # 你的代码: X1 * X2
    
    # TODO: 生成处理 T (通过 sigmoid 函数)
    # logit = Phi1 + 0.5*Phi2
    # P(T=1) = sigmoid(logit) = 1 / (1 + exp(-logit))
    logit = None  # 你的代码
    propensity = None  # 你的代码: 1 / (1 + np.exp(-logit))
    T = None  # 你的代码: np.random.binomial(1, propensity, n)
    
    # TODO: 生成结果 Y
    # Y = 1 + 2*T + Phi1 + 0.5*Phi2 + noise (noise ~ N(0, 0.5))
    noise = np.random.randn(n) * 0.5
    Y = None  # 你的代码
    
    X = np.column_stack([X1, X2])
    
    return X, T, Y

# 测试
X, T, Y = generate_nonlinear_data()
if X is not None and X[0, 0] is not None:
    print(f"数据形状: X={X.shape}, T={T.shape}, Y={Y.shape}")
    print(f"处理比例: {T.mean():.2%}")
    print(f"平均结果: {Y.mean():.4f}")
    print(f"\n真实 ATE = 2.0 (这是我们设定的)")
else:
    print("[未完成] 请完成 generate_nonlinear_data 函数")

In [ ]:
# 练习 1.1 续: 朴素线性估计

def naive_linear_estimation(X, T, Y):
    """
    使用线性回归估计 ATE (直接使用原始特征)
    
    TODO: 实现线性回归估计
    
    提示: Y = beta0 + beta1*T + beta2*X1 + beta3*X2 + epsilon
          beta1 就是 ATE 估计
    """
    # TODO: 构造特征矩阵 [T, X]
    features = None  # 你的代码: np.column_stack([T, X])
    
    if features is None:
        return None
    
    # TODO: 训练线性回归
    model = LinearRegression()
    model.fit(features, Y)
    
    # TODO: 返回 T 的系数作为 ATE 估计
    ate_estimate = model.coef_[0]  # T 是第一个特征
    
    return ate_estimate

# 测试
if X is not None:
    ate_linear = naive_linear_estimation(X, T, Y)
    if ate_linear is not None:
        print(f"线性回归估计的 ATE: {ate_linear:.4f}")
        print(f"真实 ATE: 2.0")
        print(f"误差: {abs(ate_linear - 2.0):.4f}")
        print(f"\n由于忽略了非线性关系，线性估计可能有偏差!")
    else:
        print("[未完成] 请完成 naive_linear_estimation 函数")

---

## Part 2: 简单的表示学习网络

### 神经网络架构

```
原始特征 X (2维)
     |
     ↓
[Hidden Layer] (20 neurons, ReLU)
     |
     ↓
表示 Φ(X) (10维)
```

这个网络将 2 维的原始特征转换成 10 维的 "表示"。希望这个表示能自动捕获非线性关系！

In [ ]:
# 练习 1.2: 简单的表示学习网络

class SimpleRepresentation(nn.Module):
    """
    简单的表示学习网络
    
    X -> [Hidden Layer] -> Phi(X) (学习到的表示)
    """
    
    def __init__(self, input_dim: int, repr_dim: int = 10, hidden_dim: int = 20):
        super().__init__()
        
        # TODO: 定义网络层
        # 提示: Input -> Linear -> ReLU -> Linear -> Output
        
        self.network = nn.Sequential(
            # 你的代码:
            # nn.Linear(input_dim, hidden_dim),
            # nn.ReLU(),
            # nn.Linear(hidden_dim, repr_dim),
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """前向传播"""
        return self.network(x)


def train_representation(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    repr_dim: int = 10,
    n_epochs: int = 100
) -> nn.Module:
    """
    训练表示学习网络
    
    目标: 学习能预测 Y 的表示 Phi(X)
    
    TODO: 完成训练过程
    
    训练策略:
    1. Phi(X) -> [Linear] -> Y_pred
    2. 最小化 MSE(Y, Y_pred)
    3. 学到的 Phi(X) 可用于后续因果推断
    """
    
    # 转换为 Tensor
    X_tensor = torch.FloatTensor(X)
    Y_tensor = torch.FloatTensor(Y).unsqueeze(1)
    
    # 模型
    repr_model = SimpleRepresentation(input_dim=X.shape[1], repr_dim=repr_dim)
    
    # TODO: 定义预测头 (Phi(X) -> Y)
    prediction_head = None  # 你的代码: nn.Linear(repr_dim, 1)
    
    if prediction_head is None:
        return repr_model  # 返回未训练的模型
    
    # TODO: 定义优化器和损失函数
    params = list(repr_model.parameters()) + list(prediction_head.parameters())
    optimizer = None  # 你的代码: optim.Adam(params, lr=0.01)
    criterion = None  # 你的代码: nn.MSELoss()
    
    # TODO: 训练循环
    for epoch in range(n_epochs):
        # 你的代码:
        # optimizer.zero_grad()
        # phi = repr_model(X_tensor)
        # y_pred = prediction_head(phi)
        # loss = criterion(y_pred, Y_tensor)
        # loss.backward()
        # optimizer.step()
        pass
    
    return repr_model

# 测试
if X is not None:
    repr_model = train_representation(X, T, Y, repr_dim=5, n_epochs=50)
    print("表示学习模型创建完成")
    print(f"模型结构:")
    print(repr_model)

---

## Part 3: 可视化表示空间

### 为什么要可视化?

通过可视化学到的表示，我们可以检查：

1. **表示是否有意义?** 相似的样本应该在表示空间中接近
2. **处理组和控制组是否可区分?** 如果完全分离，可能存在问题
3. **表示是否捕获了重要信息?**

In [ ]:
# 练习 1.3: 可视化表示空间

def visualize_representation(
    repr_model: nn.Module,
    X: np.ndarray,
    T: np.ndarray
) -> Tuple[np.ndarray, np.ndarray]:
    """
    可视化学到的表示空间
    
    TODO:
    1. 使用训练好的模型提取表示 Phi(X)
    2. 如果维度 > 2, 使用 PCA 降维到 2D
    3. 返回 2D 表示用于绘图
    
    Returns:
        (phi_2d, T) - 2D 表示和处理标签
    """
    
    repr_model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X)
        
        # TODO: 提取表示
        phi = None  # 你的代码: repr_model(X_tensor)
        
        if phi is None:
            return None, T
        
        phi = phi.numpy()
    
    # TODO: 如果维度 > 2, 使用 PCA 降维
    if phi.shape[1] > 2:
        # 你的代码:
        # pca = PCA(n_components=2)
        # phi = pca.fit_transform(phi)
        pass
    
    return phi, T

# 可视化
if X is not None and repr_model is not None:
    phi_2d, _ = visualize_representation(repr_model, X, T)
    
    if phi_2d is not None:
        plt.figure(figsize=(10, 4))
        
        # 原始特征空间
        plt.subplot(1, 2, 1)
        plt.scatter(X[T==0, 0], X[T==0, 1], alpha=0.5, label='Control', s=10)
        plt.scatter(X[T==1, 0], X[T==1, 1], alpha=0.5, label='Treated', s=10)
        plt.xlabel('X1')
        plt.ylabel('X2')
        plt.title('Original Feature Space')
        plt.legend()
        
        # 表示空间
        plt.subplot(1, 2, 2)
        if phi_2d.shape[1] >= 2:
            plt.scatter(phi_2d[T==0, 0], phi_2d[T==0, 1], alpha=0.5, label='Control', s=10)
            plt.scatter(phi_2d[T==1, 0], phi_2d[T==1, 1], alpha=0.5, label='Treated', s=10)
            plt.xlabel('Phi1')
            plt.ylabel('Phi2')
        plt.title('Learned Representation Space')
        plt.legend()
        
        plt.tight_layout()
        plt.show()
    else:
        print("[未完成] 请完成 visualize_representation 函数")

---

## Part 4: 表示平衡性检查

### 什么是表示平衡?

在因果推断中，我们希望处理组和控制组在表示空间中的分布尽可能相似。

如果两组分布差异太大，说明存在 **协变量不平衡**，可能导致估计偏差。

### 平衡性指标

1. **SMD (标准化均值差)**: $\frac{|\bar{\Phi}_T - \bar{\Phi}_C|}{\sigma_{\Phi}}$
   - SMD < 0.1 通常被认为是良好平衡

2. **MMD (最大均值差异)**: $||\bar{\Phi}_T - \bar{\Phi}_C||^2$
   - 衡量两组分布的整体差异

In [ ]:
# 练习 1.4: 表示平衡性检查

def check_representation_balance(
    phi: np.ndarray,
    T: np.ndarray
) -> dict:
    """
    检查表示在处理组和对照组之间的平衡性
    
    平衡性指标:
    1. 标准化均值差 (SMD): |mean(Phi_T) - mean(Phi_C)| / std(Phi)
    2. MMD (Maximum Mean Discrepancy): ||mean(Phi_T) - mean(Phi_C)||^2
    
    TODO: 计算平衡性指标
    
    Returns:
        dict with balance metrics
    """
    
    # 分离处理组和对照组
    phi_treated = phi[T == 1]
    phi_control = phi[T == 0]
    
    # TODO: 计算 SMD (对每个维度)
    # SMD_j = |mean(Phi_T[:, j]) - mean(Phi_C[:, j])| / std(Phi[:, j])
    mean_t = phi_treated.mean(axis=0)
    mean_c = phi_control.mean(axis=0)
    std_all = phi.std(axis=0) + 1e-8  # 避免除零
    
    smd = None  # 你的代码: np.abs(mean_t - mean_c) / std_all
    
    # TODO: 计算 MMD (简化版: 欧氏距离)
    # MMD = ||mean(Phi_T) - mean(Phi_C)||^2
    mmd = None  # 你的代码: np.sum((mean_t - mean_c)**2)
    
    return {
        'smd_mean': np.mean(smd) if smd is not None else None,
        'smd_max': np.max(smd) if smd is not None else None,
        'mmd': mmd
    }

# 测试
if X is not None and repr_model is not None:
    repr_model.eval()
    with torch.no_grad():
        phi = repr_model(torch.FloatTensor(X)).numpy()
    
    balance = check_representation_balance(phi, T)
    if balance['mmd'] is not None:
        print(f"平均 SMD: {balance['smd_mean']:.4f}")
        print(f"最大 SMD: {balance['smd_max']:.4f}")
        print(f"MMD: {balance['mmd']:.4f}")
        print(f"\n提示: SMD < 0.1 表示良好平衡")
    else:
        print("[未完成] 请完成 check_representation_balance 函数")

---

## Part 5: 对比实验 - 线性 vs 表示学习

让我们对比一下：使用表示学习是否能改善 ATE 估计？

In [ ]:
# 练习 1.5: 对比线性方法和表示学习

def compare_linear_vs_learned(
    X: np.ndarray,
    T: np.ndarray,
    Y: np.ndarray,
    true_ate: float = 2.0
) -> dict:
    """
    对比线性方法和表示学习方法
    
    TODO:
    1. 使用线性回归 (原始特征) 估计 ATE
    2. 训练表示学习模型
    3. 使用学到的表示估计 ATE
    4. 比较两种方法的误差
    
    Returns:
        dict with comparison results
    """
    
    # 1. 线性估计
    linear_ate = naive_linear_estimation(X, T, Y)
    
    # 2. 表示学习
    repr_model = train_representation(X, T, Y, repr_dim=10, n_epochs=200)
    
    # 3. 提取表示并估计 ATE
    repr_model.eval()
    with torch.no_grad():
        phi = repr_model(torch.FloatTensor(X)).numpy()
    
    # 用 Phi 替代 X 进行线性回归
    features_phi = np.column_stack([T, phi])
    model = LinearRegression()
    model.fit(features_phi, Y)
    learned_ate = model.coef_[0]  # T 的系数
    
    return {
        'linear_ate': linear_ate,
        'linear_error': abs(linear_ate - true_ate) if linear_ate else None,
        'learned_ate': learned_ate,
        'learned_error': abs(learned_ate - true_ate) if learned_ate else None,
        'true_ate': true_ate
    }

# 运行对比
if X is not None:
    results = compare_linear_vs_learned(X, T, Y, true_ate=2.0)
    if results['linear_ate'] is not None:
        print("=" * 40)
        print("线性方法 vs 表示学习 对比")
        print("=" * 40)
        print(f"真实 ATE: {results['true_ate']:.4f}")
        print(f"\n线性估计: {results['linear_ate']:.4f} (误差: {results['linear_error']:.4f})")
        if results['learned_ate'] is not None:
            print(f"表示学习估计: {results['learned_ate']:.4f} (误差: {results['learned_error']:.4f})")
    else:
        print("[未完成] 请完成所有前序练习")

---

## 思考题

### 1. 为什么线性模型在非线性数据上表现不好?

**你的答案:**


### 2. 表示学习如何帮助因果推断?

**你的答案:**


### 3. 什么是 "表示平衡" (Representation Balance)? 为什么重要?

**你的答案:**


### 4. 在深度因果模型中，共享表示层的作用是什么?

**你的答案:**


### 5. 如果处理组和对照组的表示分布完全不重叠，会有什么问题?

**你的答案:**


---

## 总结

| 概念 | 要点 |
|------|------|
| **表示学习** | 让神经网络自动学习有用的特征表示 |
| **非线性关系** | 原始特征的复杂组合可能才是真正有用的 |
| **表示平衡** | 处理组和控制组在表示空间中应该分布相似 |
| **SMD** | 衡量单个维度的不平衡程度 |
| **MMD** | 衡量整体分布差异 |

### 为什么这很重要?

表示学习是深度因果模型的基础。在下一个练习中，我们将学习 **TARNet** —— 第一个专门为因果推断设计的神经网络！

---

*恭喜你完成了表示学习基础的学习！* 🎉